# GeoObserver - A General Earth Engine App 


In [1]:
import os
import ee
import geemap
import calendar
import fontstyle
import pandas as pd
import ipywidgets as widgets
from datetime import datetime
from ipyleaflet import WidgetControl
from IPython.display import display
from tabulate import tabulate
from datetime import datetime
import matplotlib.pyplot as plt
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# ee.Authenticate()
ee.Initialize()

In [3]:
# NDVI - EVI - SAVI - NDBI - NDWI - NDSI 

def ndviImage(image):
    ndvi = image.normalizedDifference(["SR_B4", "SR_B3"])
    return image.addBands(ndvi.rename("NDVI"))

def ndbiImage(image):
    ndbi = image.normalizedDifference(["SR_B5", "SR_B4"])
    return image.addBands(ndbi.rename("NDBI"))

def ndwiImage(image):
    ndwi = image.normalizedDifference(["SR_B2", "SR_B4"])
    return image.addBands(ndwi.rename("NDWI"))

def ndsiImage(image):
    ndsi = image.normalizedDifference(["SR_B2", "SR_B5"])
    return image.addBands(ndsi.rename("NDSI"))

def eviImage(image) :
    evi = image.expression('2.5*((NIR - RED)/((NIR) + (6 * RED) - (7.5 * BLUE) + 1))', 
          {'NIR': image.select('SR_B4'),'RED': image.select('SR_B3'), 'BLUE': image.select('SR_B1')})
    return image.addBands(evi.rename('EVI'));

def saviImage(image) :
    nir, red = image.select('SR_B4'), image.select('SR_B3')
    numerator = (nir.subtract(red)).multiply(ee.Image(1.5)) 
    denominator = (nir.add(red)).add(ee.Image(0.5))
    savi =  (numerator.divide(denominator)).rename('SAVI')
    return image.addBands(savi)

In [4]:
current_time = ''
current_indices = ''

style = {'description_width': 'initial'}

months = list(calendar.month_name)

year = widgets.IntSlider(description='Year:',
                         value=2000,
                         min=2000,
                         max=2022,
                         style=style)
month = widgets.IntSlider(description='Month:',
                          value=1,
                          min=1,
                          max=12,
                          style=style)

nd_options = ["NDVI", 'EVI', 'NDWI', 'NDSI', 'NDBI', 'SAVI']

nd_indices = widgets.Dropdown(
    options=nd_options,
    value="NDVI",
    description='Index : ',
    style=style,
)


clr_options = [ ["red", "yellow", "green"], ["red", "orange", "yellow"], ["red", "lightsteelblue", "blue"], 
                ["blue", "green", "red"], ["red", "pink", "orange"], ["blue", "cyan", "grey"]]

plates = widgets.Dropdown(
    description='Colours',
    options=clr_options,
    value=["red", "yellow", "green"],
    style=style,
)

def nd_color_change(change):                                        
    if nd_indices.value == 'NDVI':     
        plates.value = ["red", "yellow", "green"]         
    elif nd_indices.value == 'EVI':    
        plates.value = ["red", "orange", "yellow"]
    elif nd_indices.value == 'NDWI':
        plates.value = ["red", "lightsteelblue", "blue"]
    elif nd_indices.value == 'NDSI':
        plates.value = ["blue", "green", "red"]
    elif nd_indices.value == 'NDBI':
        plates.value = ["red", "pink", "orange"]
    elif nd_indices.value == 'SAVI':
        plates.value = ["blue", "cyan", "grey"]
    else:
        plates.value = ["red", "yellow", "green"]


nd_indices.observe(nd_color_change, names='value')

compute = widgets.Button(
    description='compute',
    button_style='primary',
    tooltip='click to compute the index',
    style=style,
)


hbox = widgets.HBox([year, month, nd_indices, compute])

output = widgets.Output()

In [5]:
def submit_clicked(b):
    global items, AOI, limit, data_lis
    with output:

        output.clear_output()

        def clipImage(image):
            return image.clip(AOI)
        
        def setImage(image):
            Mean = (image.reduceRegion(reducer= ee.Reducer.mean(), geometry = AOI, scale = 30, maxPixels = 1e30)).get(nd_indices.value)
            Median = (image.reduceRegion(reducer= ee.Reducer.median(), geometry = AOI, scale = 30, maxPixels = 1e30)).get(nd_indices.value)
            return image.set({'Mean':Mean , 'Median' : Median })

        AOI = Map.user_rois

        dataset = ee.ImageCollection("LANDSAT/LE07/C02/T1_L2")\
                .filterBounds(AOI).filter(ee.Filter.calendarRange(year.value,year.value,'year'))\
                .filter(ee.Filter.calendarRange(month.value,month.value,'month'))\
                .map(ndviImage).map(eviImage).map(ndwiImage).map(ndsiImage).map(ndbiImage).map(saviImage)\
                .map(clipImage).select(nd_indices.value).map(setImage)
        
        limit = dataset.size().getInfo()
        data_lis = dataset.toList(dataset.size())
        
        dataset.propertyNames().getInfo()
        
        items = ["LANDSAT/LE07/C02/T1_L2/"+x for x in dataset.aggregate_array("system:index").getInfo()]     
        image = dataset.toBands()
            
        Mean = dataset.aggregate_array("Mean").getInfo()
        Median = dataset.aggregate_array("Median").getInfo()
        
        
        df = pd.DataFrame(list(zip(items, Mean, Median)),columns =['ID', 'Mean', 'Median'])
        
        vis_params = {
            "min": -1,
            "max": 1,
            "palette": plates.value 
        }

        Map.clear_layers()
        Map.add_basemap('HYBRID')

        Map.addLayer(dataset.select(nd_indices.value), vis_params,nd_indices.value)

        text = f"Showing {nd_indices.value} map for the selected area, for {months[month.value]} {year.value}"
        
        print(text)
        print('\nDownloading the csv file of data...')
        print("\nitems present in the collection are as follows : \n")
        print(tabulate(df, headers = 'keys', tablefmt = 'fancy_grid'))

        now = datetime.now()
        current = now.strftime("%H:%M:%S")
        
        file_name = f'indices_{nd_indices.value}_downloaded_at_time_{current}.csv'
        df.to_csv(file_name)
        current_time = current
        current_indices = nd_indices.value

        print(f'\nData is downloaded with filename : {file_name}')
        
compute.on_click(submit_clicked)

In [6]:
download = widgets.Button(
    description='download images  (.tif)',
    button_style='primary',
    tooltip='Download all the images',
    style=style,
)

hbox1 = widgets.HBox([download]) 

In [7]:
def download_images(b):
    with output:

        output.clear_output()
        for x in range(limit) :
            try :
                print("downloading : ", items[x])
                geemap.download_ee_image(ee.Image(data_lis.get(x)), f"{items[x].split('/')[-1]}.tif", scale = 30, crs = 'epsg:4326', region = AOI.geometry())
            except :
                pass
download.on_click(download_images)

In [10]:
Map = geemap.Map()
Map
hbox
hbox1

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [11]:
output

Output()